In [5]:
import re
import numpy as np
import sys
import time
from operator import add
d_corpus = sc.textFile("../data/SmallTrainingData.txt", 1)
d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

In [13]:
def buildArray (listOfIndices):
    returnVal = np.zeros (20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum (returnVal)
    returnVal = np.divide (returnVal, mysum)
    return returnVal
def oneHotEncoding(ID):
    if re.compile('^AU').match(ID):
        return 1
    else:
        return 0

In [14]:
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))
allWords = d_keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
allCounts = allWords.reduceByKey(add)
topWords = allCounts.top(20000, key=lambda x: x[1])
twentyK = sc.parallelize(range(20000))
dictionary = twentyK.map (lambda x : (topWords[x][0], x))
allWords = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
allDictionaryWords = dictionary.join(allWords)
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))


data = tfs.map(lambda x: (oneHotEncoding(x[0]), x[1]))

In [15]:


def calculateTheta(x, r_prev):
    return np.dot(x, r_prev)
def getLogPart(theta):
    return np.log(1 + np.exp(theta))
def calculateXtY(x, y):
    return -np.multiply(x, y)
def getXMultiplyThetaPart(x, theta):
    return np.multiply(x, (np.exp(theta) / (1.0 + np.exp(theta))))
def getYMultiplyTheta(y, theta):
    return -np.multiply(y, theta)
def getDerivative(x, y, theta):
    XtY = calculateXtY(x, y)
    XTheta = getXMultiplyThetaPart(x, theta)
    derivative = np.add(XtY, XTheta)
    return derivative
def getLLH(x, y, theta):
    Ytheta = getYMultiplyTheta(y, theta)
    logPart = getLogPart(theta)
    cost = np.add(Ytheta, logPart)
    return cost

In [16]:
#def gradientDescent(learningRate = 0.01, num_iteration = 100):
num_iteration = 100
learningRate = 0.01
r_prev= np.full(20000, 0.1)
count = 0
cost_prev = sys.float_info.max
precision = 0.00000001
prev_stepsize = 1
while(num_iteration > count and precision < prev_stepsize):
    start = time.time()

    # Calculate cost and derivative
    costAndDerivative = data.map(lambda x: (x[0], x[1], calculateTheta(x[1], r_prev)))\
                            .map(lambda x: (1, np.append(getLLH(x[1], x[0], x[2]), getDerivative(x[1], x[0], x[2]))))\
                            .reduceByKey(lambda x1, x2:np.add(x1, x2)).collect()
   

    cost_cur = costAndDerivative[0][1][0]
    dr = costAndDerivative[0][1][1:]


    # Set current weight 
    r_cur = np.subtract(r_prev, learningRate*dr)



    # Bold Driver logic
    if cost_cur < cost_prev:
        learningRate *= 1.05
    else:
        learningRate *= 0.5

    print("Iteration: ", count)
    print("W: " ,r_cur)
    print("Loss: ", cost_cur)

    count += 1

    # Calculate stepsize
    prev_stepsize = np.linalg.norm(np.subtract(r_prev, r_cur), 2)

    r_prev = r_cur

    cost_prev = cost_cur

    stop = time.time()
    duration = stop - start
    print("Duration", duration)

Iteration:  0
W:  [-1.257677   -0.53386255 -0.47453573 ...  0.0998765   0.09989883
  0.09994906]
Loss:  2554.813303973212
Duration 19.793058156967163
Iteration:  1
W:  [-2.53118535 -1.12872441 -1.01730984 ...  0.09975806  0.09980348
  0.09990184]
Loss:  2219.4747471804358
Duration 1.6604039669036865
Iteration:  2
W:  [-3.72016817 -1.68441814 -1.52818688 ...  0.09964469  0.09971393
  0.09985833]
Loss:  1937.949624474074
Duration 1.6955928802490234
Iteration:  3
W:  [-4.82756221 -2.20230046 -2.0082759  ...  0.09953625  0.09962991
  0.09981837]
Loss:  1703.4106461335032
Duration 1.634948968887329
Iteration:  4
W:  [-5.85838035 -2.68468924 -2.45947868 ...  0.09943244  0.09955099
  0.0997817 ]
Loss:  1508.6892589592753
Duration 1.6234228610992432
Iteration:  5
W:  [-6.81867617 -3.13438452 -2.88410461 ...  0.09933289  0.09947668
  0.099748  ]
Loss:  1347.0403223294454
Duration 2.3196029663085938
Iteration:  6
W:  [-7.71479925 -3.55432406 -3.28459174 ...  0.09923723  0.09940645
  0.09971697]


Iteration:  53
W:  [-22.41486977 -10.5681983  -14.0466179  ...   0.09533558   0.09692188
   0.09899879]
Loss:  387.82745892086024
Duration 1.4819178581237793
Iteration:  54
W:  [-22.38645276 -10.55932077 -14.27068116 ...   0.09521107   0.09684581
   0.09898132]
Loss:  386.65486035604005
Duration 1.531977891921997
Iteration:  55
W:  [-22.34518806 -10.5445045  -14.50023341 ...   0.09508194   0.09676691
   0.09896318]
Loss:  385.4578601589598
Duration 1.5228192806243896
Iteration:  56
W:  [-22.29118852 -10.52380122 -14.73582883 ...   0.09494789   0.09668501
   0.09894433]
Loss:  384.2299273555981
Duration 1.5220177173614502
Iteration:  57
W:  [-22.22459007 -10.49727271 -14.97805102 ...   0.09480863   0.09659991
   0.0989247 ]
Loss:  382.96506129065125
Duration 1.5535368919372559
Iteration:  58
W:  [-22.14555076 -10.46499027 -15.22751281 ...   0.09466381   0.09651139
   0.09890423]
Loss:  381.6577414459527
Duration 1.6055819988250732
Iteration:  59
W:  [-22.05424914 -10.4270338  -15.484855

In [73]:
test_corpus = sc.textFile("../data/TestingData.txt", 1)
test_keyAndText = test_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
test_keyAndListOfWords = test_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [74]:
test_allWords = test_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))
test_allDictionaryWords = dictionary.join(test_allWords)
test_justDocAndPos = test_allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))
test_allDictionaryWordsInEachDoc = test_justDocAndPos.groupByKey()
test_tfs = test_allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))
test = test_tfs.map(lambda x: (oneHotEncoding(x[0]), x[1], x[0]))

In [80]:
prediction = test.map(lambda x: (x[0], np.dot(x[1], r_cur), x[2]))\
                 .map(lambda x: (x[0], (1 if x[1] >= 0 else 0), x[2]))

In [81]:
prediction.take(5)

[(1, 0, 'AU67'),
 (1, 0, 'AU87'),
 (1, 0, 'AU329'),
 (1, 0, 'AU360'),
 (1, 0, 'AU365')]

In [92]:
def getType1AndType2(true, pred):
    if true == 1 and pred == 1:
        TP = 1
    else:
        TP = 0
    if true == 0 and pred == 0:
        TN = 1
    else:
        TN = 0
    if true == 1 and pred == 0:
        FN = 1
    else:
        FN = 0
    if true == 0 and pred == 1:
        FP = 1
    else:
        FP = 0
    
    return np.array([TP, TN, FN, FP])

In [93]:
type1AndType2 = prediction.map(lambda x: (x[2], getType1AndType2(x[0], x[1])))
matric = type1AndType2.map(lambda x: (1, x[1])).reduceByKey(lambda x1, x2: np.add(x1, x2)).collect()

In [94]:
type1AndType2.take(5)

[('AU67', array([0, 0, 1, 0])),
 ('AU87', array([0, 0, 1, 0])),
 ('AU329', array([0, 0, 1, 0])),
 ('AU360', array([0, 0, 1, 0])),
 ('AU365', array([0, 0, 1, 0]))]

In [95]:
TP = matric[0][1][0]
TN = matric[0][1][1]
FN = matric[0][1][2]
FP = matric[0][1][3]

In [96]:
def getF1score(TP, TN, FN, FP):
    if TP == 0:
        return 0
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f_score = 2 * (precision * recall) / (precision + recall)
    return f_score

In [97]:
f1_score = getF1score(TP, TN, FN, FP)

In [99]:
f1_score

0

In [127]:
docFP = type1AndType2.filter(lambda x: x[1][3] == 1).take(3)

In [128]:
docFPID = [d[0] for d in docFP]

In [129]:
docFPID

['31617911']

In [136]:
pickedDoc = test_keyAndText.filter(lambda x: x[0] in docFPID)

In [137]:
pickedDoc.saveAsTextFile("output2")

In [109]:
pickedDoc = test_allDictionaryWordsInEachDoc.map(lambda x: (x[0], np.array(list(x[1])))).take(3)

In [111]:
pickedDocRDD = sc.parallelize(pickedDoc)

In [112]:
pickedDocRDD.take(1)

[('AU67', array([    6,     6,     6, ...,  8904, 11655, 15532]))]

In [103]:
dictionary.take(5)

[('the', 0), ('of', 1), ('and', 2), ('in', 3), ('to', 4)]

[('AU11',
  "purported appeal from orders made by federal magistrate dismissing applicant's application for review by refugee review tribunal of decision of minister's delegate refusing applicant protection visadecision of minister's delegate affirmed by refugee review tribunalobligations of tribunal under ss 424a and 425 of the migration act 1958 whether decision of magistrate interlocutory in natureeffect of federal magistrates court rules 2001  r 44.12whether leave to appeal required under federal court of australia act 1976  s 24.migrationprocedureThis is a purported appeal from a decision of a Federal Magistrate.The Magistrate made an order dismissing an application under s 476 of the Migration Act 1958  ('the Act').The application to the Federal Magistrates Court under s 476 of the Act was supported by an affidavit as required by r 44.05 of the Federal Magistrates Court Rules 2001 and it came before the Magistrate on a first court date on 30 January 2006.At that time, it was not 